# Project 5: PageRank and Solving Linear Systems with Jacobi Iterations and CGD

**Your name here:**    <i>Kevin Wong</i>

You can use basic functions from numpy. When I ask you to define a function, you need to write it from scratch. You are free to copy code from my notebooks.

In [2]:
import numpy as np
import timeit

## Part 1: PageRank algorithm

1 (1pt) Run the following command to load this data. This is more than 1000 blogs and how they are linked to each other.

Source: Lada A. Adamic and Natalie Glance, "The political blogosphere and the 2004 US Election", in Proceedings of the WWW-2005 Workshop on the Weblogging Ecosystem (2005)

In [2]:
blog = np.loadtxt('blog.csv', delimiter=',')

In [3]:
# blog is now a numpy array. You can print a few rows to see the structure:
print blog[:30,:]

[[  267.  1394.]
 [  267.   483.]
 [  267.  1051.]
 [  904.  1479.]
 [  904.   919.]
 [  904.  1045.]
 [  904.  1330.]
 [  904.  1108.]
 [  904.   995.]
 [  904.   963.]
 [  904.  1000.]
 [  904.  1192.]
 [  904.  1067.]
 [  904.  1270.]
 [  904.  1437.]
 [  904.  1037.]
 [  903.   855.]
 [  903.  1008.]
 [  982.  1429.]
 [  982.   952.]
 [  982.   892.]
 [  982.  1479.]
 [  982.   956.]
 [  982.  1306.]
 [  982.   810.]
 [  982.  1437.]
 [  982.  1112.]
 [  982.  1051.]
 [  982.   826.]
 [  982.  1478.]]


This is saying that, for example, Blog 903 has links to blog 855 and blog 1008.

2 (3pts) We need to generate a binary square matrix indicating outgoing pages. Recall that for such a matrix, column j indicating the links that blog j has. I've written the code for you since this is not a data processing class.

In [4]:
#Get the biggest and smallest blog post ID
N_max = np.max(blog)
N_min = np.min(blog)
print N_max, N_min
n = N_max - N_min + 1 #this is the size of the binary matrix
n = int(n)

1490.0 1.0


In [5]:
# This binary matrix will be called BM (short for blog matrix)
BM = np.zeros([n,n])
for i in range(blog.shape[0]): #going through each row of the original data
    blogID = blog[i,0] 
    OutB = blog[i,1]
    BM[int(OutB - 1),int(blogID -1)] = 1.0 #-1 is because the blog ID starts at 1, but python starts at 0

In [6]:
# test: as mentioned earlier blog 903 has linkes to blog 855 and blog 1008. 
# This would mean the two entries below are 1
print BM[854,902], BM[1007,902]

1.0 1.0


 3 (4pts) Check if there is a dangling page: a page that does not have any outgoing links. If there is, alter that column to represent a self-link.

In [7]:
# Confirming that BM is sqaure matrix
print BM.shape[0], BM.shape[1]

1490 1490


In [28]:
# 159 pages don't have any outgoing links

dangleCount = 0

for i in range(BM.shape[0]):
    hasOutgoing = False
    
    # Iterate down the column, checking for outgoing links
    for j in range(BM.shape[1]):
        if(BM[j][i] == 1.0):
            hasOutgoing = True
    
    # Updating the counter and setting a self-link
    if(not hasOutgoing):
        dangleCount += 1
        BM[i][i] = 1.0

print dangleCount

159


In [29]:
# After altering to represent an outgoing link to itself, there are 0 dangling pages

dangleCount2 = 0

for i in range(BM.shape[0]):
    hasOutgoing = False
    for j in range(BM.shape[1]):
        if(BM[i][j] == 1.0):
            hasOutgoing = True
            
    if(not hasOutgoing):
        dangleCount2 += 1

print dangleCount2

0


4 (3pts) Normalize each column such that each column sums to 1. Now we have obtained a probability transition matrix BMP.

In [30]:
# A list representing the number of outgoing edges from each blog
outgoingSum = np.sum(BM, axis = 0)
outgoingSum

array([ 15.,  43.,   1., ...,   2.,   8.,   2.])

In [31]:
# Confirming that all blogs have an outgoing edge now
0 in outgoingSum

False

In [32]:
BM

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [33]:
BMP = BM/np.outer(np.ones(BM.shape[0]), outgoingSum)

In [36]:
# Probabibility transition matrix
BMP

array([[ 0.        ,  0.02325581,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.5       ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.5       ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.5       ]])

5 (2pts) Define another probability transition matrix M to allow random visits to other blogs. Pick $\delta=0.85$.

In [37]:
# Delta represents the probability of choosing a link from the blog page we are on
delta = .85

# M incorporates the idea that we can jump to any blog with (1 - delta) probability
M = delta*BMP + ((1-delta) / n) * np.outer(np.ones(BMP.shape[0]),np.ones(BMP.shape[0]))

In [38]:
M

array([[  1.00671141e-04,   1.98681130e-02,   1.00671141e-04, ...,
          1.00671141e-04,   1.00671141e-04,   1.00671141e-04],
       [  1.00671141e-04,   1.00671141e-04,   1.00671141e-04, ...,
          1.00671141e-04,   1.00671141e-04,   1.00671141e-04],
       [  1.00671141e-04,   1.00671141e-04,   8.50100671e-01, ...,
          1.00671141e-04,   1.00671141e-04,   1.00671141e-04],
       ..., 
       [  1.00671141e-04,   1.00671141e-04,   1.00671141e-04, ...,
          4.25100671e-01,   1.00671141e-04,   1.00671141e-04],
       [  1.00671141e-04,   1.00671141e-04,   1.00671141e-04, ...,
          4.25100671e-01,   1.00671141e-04,   1.00671141e-04],
       [  1.00671141e-04,   1.00671141e-04,   1.00671141e-04, ...,
          1.00671141e-04,   1.00671141e-04,   4.25100671e-01]])

6 (6pts) Use the power method to find the eigenvector (corresponding to the biggest eigenvalue) of M. Name this vector RV (short for ranking vector). Your algorithm terminates when RV[i] is changing less then $\epsilon=10^{-10}$ for each entry i. Remember to start with a vector whose entries are all nonnegative and entries sum to 1. 

In [58]:
# Power method function with custom termination condition, returns the ranking vector
# The ranking vector represents which pages are "more important"
def Anx(A,x0): 
    x = x0
    terminate = False
    while(not terminate):
        x_prev = x
        x = np.dot(A,x)
        if all(np.abs(x-x_prev) < 1e-10):
            terminate = True
            
    return x

In [57]:
# Initialize a starting vector 
x0 = np.ones(M.shape[0])/M.shape[0]

# Using the power method with termination condition on the starting vector to find the ranking vector
RV = Anx(M, x0)
print RV

[ 0.00018325  0.00018311  0.00067114 ...,  0.00017508  0.00043178
  0.00017508]


In [47]:
# Finding the ranking vector through eigenvalue/eigenvectors of matrix M
e,v = np.linalg.eig(M)
e

array([ 1.0000+0.j, -0.8500+0.j,  0.7753+0.j, ...,  0.8500+0.j,
        0.8500+0.j,  0.8500+0.j])

In [59]:
# This is the same ranking vector as the one found through the power method!
v[:,0]/sum(v[:,0])

array([ 0.00018325-0.j,  0.00018311-0.j,  0.00067114-0.j, ...,
        0.00017508-0.j,  0.00043178-0.j,  0.00017508-0.j])

7 (4pts) List the top 10 blog ID's. np.argsort might be useful.

In [125]:
# For comparison
np.mean(RV)

0.00067114093959733839

In [126]:
# The most "popular" page
np.max(RV)

0.030362245646177701

In [128]:
# The index corresponding to the most popular page
np.argmax(RV)

797

In [148]:
# The top 10 most popular pages, adjusted for correct blog index in the blog matrix 
for i in np.argsort(-RV)[:10]:
    print "Blog #" + str(i+1) + " --- " + str(RV[i])

Blog #798 --- 0.0303622456462
Blog #990 --- 0.0213360571022
Blog #1067 --- 0.0185183079035
Blog #514 --- 0.0183109888623
Blog #1086 --- 0.0181374889575
Blog #233 --- 0.013384383554
Blog #1122 --- 0.0126842590903
Blog #979 --- 0.0115481674122
Blog #996 --- 0.0113412860744
Blog #154 --- 0.0112196459035


8 (2pts) List the top 10 blog ID's when $\delta = 0.75$.

In [150]:
# Adjusting delta and observing a slight change in results
delta = .75

M = delta*BMP + ((1-delta) / n) * np.outer(np.ones(BMP.shape[0]),np.ones(BMP.shape[0]))
RV = Anx(M, x0)
for i in np.argsort(-RV)[:10]:
    print "Blog #" + str(i+1) + " --- " + str(RV[i])

Blog #798 --- 0.0192455149453
Blog #990 --- 0.0138910497666
Blog #1067 --- 0.0119997005676
Blog #514 --- 0.0117657947364
Blog #1086 --- 0.011284780716
Blog #155 --- 0.010341238177
Blog #979 --- 0.00933566945987
Blog #996 --- 0.00908912052391
Blog #1122 --- 0.00838023903098
Blog #55 --- 0.00828258927002


## Part 2: Solving Ax = b


In [23]:
# run this code that you've seen in class
def CnstrPD(n, a):
    RM = np.random.randn(n,n) 
    q,r = np.linalg.qr(RM)
    z = (np.random.rand(n)+a)
    A = q.dot(np.diag(z)).dot(q.T)
    return A

In [24]:
# run this cell to construct A,b. It took about 15 seconds on my laptop
A = CnstrPD(5000, 0.1)
b = np.random.randn(5000)

1 (4pts) Explain what kind of matrix CnstrPD(5000,0.1) will generate and what the worse (biggest) condition number can be.

Write your answer for 1 here:

*A is a matrix similar to Z, and diagonalized by orthonormal matrix Q.  Therefore, it is both positive definite AND symmetric.  The worst condition number, k(A), since A is invertible is its biggest singular value over its smallest.  Z is symmetric, and represents the eigenvalues of A, so A's singular values are the absolute values of the diagonal values ranging from .1 to 5000.1.  Thereore the worst condition number is 5000.1 / .1  = 50,000.1*

2 (3pts) Solve Ax=b using Gauss elimination. Call the solution x_g. Print the first 10 entries of x_g and print the running time.

In [33]:
start = timeit.time.time()
x_g = np.linalg.solve(A, b)
end = timeit.time.time()

print "Solved Ax = b using Gauss elimination in " + str(end-start) + " seconds"
print "First ten entries: "
print x_g[:10]

Solved Ax = b using Gauss elimination in 2.06344485283 seconds
First ten entries: 
[ 1.31393705 -0.10589696 -1.37506218 -1.13309478  6.33790306  2.91353598
 -2.32231509  1.80470705 -1.4789492  -1.75867477]


3 (5pts) Write a function Jacobi(A,b,e) to solve Ax=b. e is the tolerance level. Your algorithm terminates when x[i] is changing less than e for each entry.

In [94]:
def get_diag(M):
    return np.identity(M.shape[0]) * np.diag(M)

def Jacobi(A, b, e):
    
    diagonal_A = get_diag(A)
    x = np.linalg.solve(diagonal_A, b)
    
    terminate = False
    while(not terminate):

        Ax = np.dot(A, x)
        residual = b - Ax
        error = np.linalg.solve(diagonal_A, residual)
        x_next = x + error
        if all(np.abs(x-x_next) < e):
            terminate = True
            
        x = x_next
        
    return x_next

4 (2pts) Solve Ax=b using Jacobi iteration with e=$10^{-8}$. Call the solution x_j. Print the first 10 entries of x_j and print the running time.

In [76]:
start = timeit.time.time()
x_j = Jacobi(A, b, 1e-8)
end = timeit.time.time()

print "Solved Ax = b using Jacobi iteration in " + str(end-start) + " seconds"
print "First ten entries: "
print x_j[:10]

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100
Solved Ax = b using Jacobi iteration in 242.990867138 seconds
First ten entries: 
[ 1.31393705 -0.10589696 -1.37506218 -1.13309478  6.33790305  2.91353597
 -2.32231509  1.80470705 -1.4789492  -1.75867477]


5 (5pts) Write a function CGD(A,b,e) to solve Ax=b. e is the tolerance level. Your algorithm terminates when x[i] is changing less than e for each entry.

In [95]:
def CGD(A, b, e):
    
    # Initializing 
    x = np.zeros(A.shape[1])
    r = b - np.dot(A, x)
    p = r
    
    terminate = False
    while(not terminate):

        Ap = np.dot(A,p)
        t = np.dot(p.T, r) / np.dot(p.T, A).dot(p)
        x_next = x + t*p
        r = r - t*Ap
        a = -(np.dot(r.T, Ap)) / np.dot(p.T, Ap)
        p = r + a*p
        if all(np.abs(x-x_next) < e):
            terminate = True
            
        x = x_next
    
    return x_next

6 (2pts) Solve Ax=b using CGD with e=$10^{-8}$. Call the solution x_cg. Print the first 10 entries of x_cg and print the running time.

In [96]:
start = timeit.time.time()
x_cg = CGD(A, b, 1e-8)
end = timeit.time.time()

print "Solved Ax = b using Conjugate Gradient Descent in " + str(end-start) + " seconds"
print "First ten entries: "
print x_cg[:10]

Solved Ax = b using Conjugate Gradient Descent in 0.872001171112 seconds
First ten entries: 
[ 1.31393705 -0.10589696 -1.37506218 -1.13309478  6.33790306  2.91353598
 -2.32231509  1.80470705 -1.47894921 -1.75867477]


7 (4pts) Run Jacobi, and CGD for other different tolerence levels and list your result in this table below.

In [97]:
errors = [1e-5, 1e-6, 1e-7]
for e in errors:
    start = timeit.time.time()
    x_jacobi = Jacobi(A, b, e)
    end = timeit.time.time()
    print "Jacobi with error " + str(e) + ": " + str(end-start)
    
    start = timeit.time.time()
    x_CD = CGD(A, b, e)
    end = timeit.time.time()
    print "CGD with error " + str(e) + ": " + str(end-start)
    

Jacobi with error 1e-05: 144.878695011
CGD with error 1e-05: 0.719403982162
Jacobi with error 1e-06: 174.719483852
CGD with error 1e-06: 0.714285135269
Jacobi with error 1e-07: 202.818871021
CGD with error 1e-07: 0.87762093544


Record running time (rounded to 3 decimals) in this table: 

||e=$10^{-5}$ | e=$10^{-6}$ | e=$10^{-7}$ | e=$10^{-8}$|
|:---|:--- | :---  | :---  | :---  |
|Jacobi|144.879|174.719|202.819|242.991|
|CGD|0.719|0.714|0.878|0.872|
|Gaussian elimination|2.063|2.063|2.063|2.063|